In [ ]:
# TRAIN & SAVE MODEL
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib

# Simulated data
np.random.seed(42)
n_samples = 500
data = pd.DataFrame({
    'temperature': np.random.normal(70, 5, n_samples),
    'vibration': np.random.normal(0.5, 0.1, n_samples),
    'operating_hours': np.random.randint(0, 1000, n_samples),
    'maintenance_needed': np.random.choice([0,1], n_samples, p=[0.8,0.2])
})

X = data.drop("maintenance_needed", axis=1)
y = data["maintenance_needed"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train model
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

# Save model in Colab session
joblib.dump(model, "rf_model.pkl")
print("✅ Model trained and saved as rf_model.pkl")


✅ Model trained and saved as rf_model.pkl


In [ ]:
# LOAD MODEL
import joblib

MODEL_PATH = "rf_model.pkl"
model = joblib.load(MODEL_PATH)
print("✅ Model loaded successfully")


✅ Model loaded successfully


In [ ]:
# Install required packages
!pip install fastapi nest-asyncio -q

from fastapi import FastAPI
from pydantic import BaseModel
import joblib
from fastapi.testclient import TestClient

# Load the model
model = joblib.load("rf_model.pkl")  # تأكد أنه موجود

# Define API and schemas
app = FastAPI(title="Predictive Maintenance API")

class MaintenanceRequest(BaseModel):
    temperature: float
    vibration: float
    operating_hours: int

class MaintenanceResponse(BaseModel):
    maintenance_needed: int

@app.post("/predict", response_model=MaintenanceResponse)
def predict(data: MaintenanceRequest):
    features = [[data.temperature, data.vibration, data.operating_hours]]
    prediction = model.predict(features)[0]
    return {"maintenance_needed": int(prediction)}

# Use TestClient to test POST requests inside Colab without running Uvicorn

client = TestClient(app)

data = {
    "temperature": 72,
    "vibration": 0.55,
    "operating_hours": 500
}

response = client.post("/predict", json=data)
print("POST request response:", response.json())



POST request response: {'maintenance_needed': 0}


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
